In [1]:
#class and imports

In [2]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

class LstmModel:
    def __init__(self, ticker):
        self.stock_symbol = ticker
        self.data = None
        self.normalizer = MinMaxScaler(feature_range=(0, 1))
        self.ds_scaled = None
        self.train_size = None
        self.test_size = None
        self.ds_train = None
        self.ds_test = None
        self.time_stamp = 100
        self.X_train = None
        self.y_train = None
        self.X_test = None
        self.y_test = None
        self.model = None

    def download_data(self, start_date, end_date):
        self.data = yf.download(tickers=self.stock_symbol, start=start_date, end=end_date, interval='1d')

    def preprocess_data(self):
        self.ds_scaled = self.normalizer.fit_transform(np.array(self.data['Close']).reshape(-1, 1))
        self.train_size = int(len(self.ds_scaled) * 0.70)
        self.test_size = len(self.ds_scaled) - self.train_size
        self.ds_train, self.ds_test = self.ds_scaled[0:self.train_size, :], self.ds_scaled[self.train_size:len(self.ds_scaled), :1]
        self.X_train, self.y_train = self.create_ds(self.ds_train, self.time_stamp)
        self.X_test, self.y_test = self.create_ds(self.ds_test, self.time_stamp)
        self.X_train = self.X_train.reshape(self.X_train.shape[0], self.X_train.shape[1], 1)
        self.X_test = self.X_test.reshape(self.X_test.shape[0], self.X_test.shape[1], 1)

    def build_model(self):
        self.model = Sequential()
        self.model.add(LSTM(units=50, return_sequences=True, input_shape=(self.X_train.shape[1], 1)))
        self.model.add(LSTM(units=50, return_sequences=True))
        self.model.add(LSTM(units=50))
        self.model.add(Dense(units=1, activation='linear'))
        self.model.compile(loss='mean_squared_error', optimizer='adam')

    def train_model(self):
        self.model.fit(self.X_train, self.y_train, validation_data=(self.X_test, self.y_test), epochs=100, batch_size=64)

    def predict_future(self):
        train_predict = self.model.predict(self.X_train)
        test_predict = self.model.predict(self.X_test)
        train_predict = self.normalizer.inverse_transform(train_predict)
        test_predict = self.normalizer.inverse_transform(test_predict)
        test = np.vstack((train_predict, test_predict))

        fut_inp = self.ds_test[278:]
        fut_inp = fut_inp.reshape(1, -1)
        tmp_inp = list(fut_inp)
        tmp_inp = tmp_inp[0].tolist()

        lst_output = []
        n_steps = 100
        i = 0
        while i < 30:
            if len(tmp_inp) > 100:
                fut_inp = np.array(tmp_inp[1:])
                fut_inp = fut_inp.reshape(1, -1)
                fut_inp = fut_inp.reshape((1, n_steps, 1))
                yhat = self.model.predict(fut_inp, verbose=0)
                tmp_inp.extend(yhat[0].tolist())
                tmp_inp = tmp_inp[1:]
                lst_output.extend(yhat.tolist())
                i += 1
            else:
                fut_inp = fut_inp.reshape((1, n_steps, 1))
                yhat = self.model.predict(fut_inp, verbose=0)
                tmp_inp.extend(yhat[0].tolist())
                lst_output.extend(yhat.tolist())
                i += 1

        ds_new = self.ds_scaled.tolist()
        ds_new.extend(lst_output)
        final_graph = self.normalizer.inverse_transform(ds_new).tolist()

        # plt.plot(final_graph,)
        # plt.ylabel("Price")
        # plt.xlabel("Time")
        # plt.title("{0} prediction of next month open".format(self.stock_symbol))
        # plt.axhline(y=final_graph[len(final_graph)-1], color = 'red', linestyle = ':', label = 'NEXT 30D: {0}'.format(round(float(*final_graph[len(final_graph)-1]),2)))
        # plt.legend()
        # plt.show() #Should comment this line for the final model
        return round(float(*final_graph[len(final_graph)-1]),2)

    @staticmethod
    def create_ds(dataset, step):
        Xtrain, Ytrain = [], []
        for i in range(len(dataset) - step - 1):
            a = dataset[i:(i + step), 0]
            Xtrain.append(a)
            Ytrain.append(dataset[i + step, 0])
        return np.array(Xtrain), np.array(Ytrain)




/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [5]:
from datetime import datetime, timedelta
import pandas as pd

def main():

    tickers = ['CMG', 'MCD', 'DPZ', 'QSR', 'KRUS', 'JACK']

    start_date = datetime(2014, 5, 15)
    end_date = datetime(2024, 2, 23)




    for ticker in tickers:
      print(f"\n------Processing {ticker}")
      predictions = pd.DataFrame()

      train_start = start_date - timedelta(days=5*365)


      if ticker == 'QSR':
        start_date = datetime(2015, 12, 15)
        train_start = start_date - timedelta(days=1*365)

      elif ticker == 'KRUS':
        start_date = datetime(2020, 8, 2)
        train_start = start_date - timedelta(days=1*365)

      current_date = start_date
      days_since = 0

      loop_start_time = datetime.now()  # Start timer


      while current_date <= end_date:

        try:
          model = LstmModel(ticker)
          model.download_data(start_date=train_start.strftime('%Y-%m-%d'), end_date=current_date.strftime('%Y-%m-%d'))
          model.preprocess_data()
          model.build_model()
          model.train_model()
          prediction_next = model.predict_future()

          # Store the prediction
          predictions = predictions.append({
              'Date': current_date,
              'Ticker': ticker,
              'Predicted Price': prediction_next
          }, ignore_index=True)
        except Exception as e:
          print(f"Error processing {ticker} on {current_date}: {e}")
        current_date = current_date + timedelta(days = 1)

      predictions.to_csv(ticker + 'data.csv')
      loop_end_time = datetime.now()  # End timer
      duration = loop_end_time - loop_start_time
      print(f"--------Completed processing {ticker}. Duration: {duration}------")


if __name__ == '__main__':
    main()


[*********************100%%**********************]  1 of 1 completed


------Processing CMG-------


Epoch 1/100
13/13 [==============================] - 25s 2s/step - loss: 0.0467 - val_loss: 0.0591
Epoch 2/100
13/13 [==============================] - 3s 198ms/step - loss: 0.0095 - val_loss: 0.0030
Epoch 3/100
13/13 [==============================] - 3s 222ms/step - loss: 0.0034 - val_loss: 0.0104
Epoch 4/100
13/13 [==============================] - 3s 251ms/step - loss: 0.0015 - val_loss: 0.0072
Epoch 5/100
13/13 [==============================] - 3s 198ms/step - loss: 0.0014 - val_loss: 0.0044
Epoch 6/100
13/13 [==============================] - 3s 195ms/step - loss: 0.0012 - val_loss: 0.0037
Epoch 7/100
13/13 [==============================] - 2s 193ms/step - loss: 0.0011 - val_loss: 0.0032
Epoch 8/100
13/13 [==============================] - 3s 262ms/step - loss: 0.0011 - val_loss: 0.0026
Epoch 9/100
13/13 [==============================] - 3s 213ms/step - loss: 0.0012 - val_loss: 0.0026
Epoch 10/100
13/13 [==============================] - 3s 196ms/step - loss: 0.0012 - val_loss

KeyboardInterrupt: 